In [40]:
import duckdb
import pandas as pd
import sqlalchemy

In [41]:
import sqlalchemy
import pandas as pd

# Crie o engine para o DuckDB
engine = sqlalchemy.create_engine('duckdb:///:memory:')

# Sua consulta SQL
sql = """
    SELECT DIA_EXTRACAO, 
    max(TOTAL_VISUALIZACOES) AS TOTAL_VISUALIZACOES_ACC
    FROM read_parquet('/home/rodrigo/Documentos/projetos/analise_dados_youtube_segunda_versao/datalake/ouro/estatisticas_videos/*/*/*/*/*/*/*/*.parquet')
    WHERE ASSUNTO='linux'
    AND ID_VIDEO IN ('U2iVEkrsJis')
    GROUP BY DIA_EXTRACAO
    ORDER BY DIA_EXTRACAO
"""

# Exemplo de criação de uma tabela e inserção de dados
with engine.connect() as connection:
    # Executar a consulta e carregar os dados em um DataFrame
    df = pd.read_sql(sql, connection)

print(df)

   DIA_EXTRACAO  TOTAL_VISUALIZACOES_ACC
0            16                      275
1            17                      279
2            18                      283
3            20                      282
4            22                      283


In [42]:
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
from sqlalchemy import text


engine = create_engine(
    'duckdb:///my_database.duckdb',
    poolclass=NullPool
)

In [43]:
nome_coluna = 'TOTAL_VISUALIZACOES'

In [44]:
sql = f"""
    SELECT DIA_EXTRACAO, 
	max({nome_coluna}) AS TOTAL_VISUALIZACOES_ACC
    FROM read_parquet('/home/rodrigo/Documentos/projetos/analise_dados_youtube_segunda_versao/datalake/ouro/estatisticas_videos/*/*/*/*/*/*/*/*.parquet')
    WHERE ASSUNTO='elder_ring'
    AND ID_VIDEO IN ('wlgI9eu4XoE')
    GROUP BY DIA_EXTRACAO
    ORDER BY DIA_EXTRACAO;
"""

In [45]:
t = text(sql)
result = pd.read_sql(t, con=engine)
result

,DIA_EXTRACAO,TOTAL_VISUALIZACOES_ACC
0,16,31122
1,17,50560
2,18,53390
3,19,54367
4,20,54841
5,21,55253
6,22,55691
7,23,55864


In [46]:
result[f'{nome_coluna}_DESLO'] = result[f'{nome_coluna}_ACC'].shift(1)
result.fillna(0, inplace=True)
result[f'{nome_coluna}_DIA'] = result[f'{nome_coluna}_ACC'] - \
    result[f'{nome_coluna}_DESLO']
result

,DIA_EXTRACAO,TOTAL_VISUALIZACOES_ACC,TOTAL_VISUALIZACOES_DESLO,TOTAL_VISUALIZACOES_DIA
0,16,31122,0.0,31122.0
1,17,50560,31122.0,19438.0
2,18,53390,50560.0,2830.0
3,19,54367,53390.0,977.0
4,20,54841,54367.0,474.0
5,21,55253,54841.0,412.0
6,22,55691,55253.0,438.0
7,23,55864,55691.0,173.0


In [47]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DIA_EXTRACAO               8 non-null      int64  
 1   TOTAL_VISUALIZACOES_ACC    8 non-null      int64  
 2   TOTAL_VISUALIZACOES_DESLO  8 non-null      float64
 3   TOTAL_VISUALIZACOES_DIA    8 non-null      float64
dtypes: float64(2), int64(2)
memory usage: 384.0 bytes
